In [1]:
import json
import pandas as pd
import google.generativeai as palm

In [2]:
from utils import VectorEmbeddings

In [3]:
with open('../.api_key', 'r') as file:
    api_key = file.readline()

In [4]:
vector_embeddings = VectorEmbeddings(api_key=api_key)

In [5]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console. Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [6]:
documents_dict = {}
document_names = ['document_1','document_2','document_3']
for document in zip(documents,document_names):
    chunks = vector_embeddings.split_text([document[0]])
    chunk_list = []
    for c in chunks:
        chunk_list.append(c.page_content)
    documents_dict[document[1]] = chunk_list

In [7]:
vector_data = []
for document, texts in documents_dict.items():
    for index, text in enumerate(texts):
        vector_data.append([document, index, text])

vector_df = pd.DataFrame(vector_data, columns=['document', 'index', 'text'])

In [8]:
vector_df['embeddings'] = vector_df['text'].apply(vector_embeddings.embed_fn)
vector_df.head()

,document,index,text,embeddings
0,document_1,0,Operating the Climate Control System Your Goo...,"[-0.004682993, 0.0658743, 0.034553137, 0.04838..."
1,document_1,1,that allows you to adjust the temperature and ...,"[0.0059845652, 0.066699535, 0.0024247267, 0.05..."
2,document_1,2,"the climate control system, use the buttons an...","[0.020519787, 0.051637597, -0.015418134, 0.062..."
3,document_1,3,Temperature: The temperature knob controls the...,"[0.02814844, 0.04172512, 0.0101980055, 0.04945..."
4,document_1,4,the knob clockwise to increase the temperature...,"[0.008129629, 0.027632985, 0.0049550766, 0.039..."


In [9]:
query = "How do you shift gears and what positions are there?"
passage = vector_embeddings.find_top_n(query=query, df = vector_df,text_col='text',n=5)
passage

,score,text
0,0.729427,move the shift lever to the desired position. ...
1,0.717824,Shifting Gears Your Googlecar has an automati...
2,0.698512,position is used to back up. Neutral: This pos...
3,0.665312,the gas pedal. Drive: This position is used to...
4,0.638044,desired mode. The available modes are: Auto: T...


In [10]:
leading_text = 'You are a helpful and informative bot that answers questions using text from the reference passage included below. Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. However, you are talking to a non-technical audience, so be sure to break down complicated concepts and strike a friendly and converstional tone. \
    If the passage is irrelevant to the answer, you may ignore it. Also, please keep it brief.'
prompt = vector_embeddings.make_prompt(leading_text, query, " ".join([i[1]['text'] for i in passage.iterrows()]))
prompt

"\n'You are a helpful and informative bot that answers questions using text from the reference passage included below. Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. However, you are talking to a non-technical audience, so be sure to break down complicated concepts and strike a friendly and converstional tone.     If the passage is irrelevant to the answer, you may ignore it. Also, please keep it brief.'\nQUESTION: 'How do you shift gears and what positions are there?'\nPASSAGE: 'move the shift lever to the desired position.  Park: This position is used when you are parked. The Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift position is used to back up. Neutral: This position is used when you are stopped at a light or in the gas pedal. Drive: This position is used to drive forward. Low: This position is used for desired mode. The available modes are: Auto: The car will au

In [11]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
text_model = text_models[0]
text_model

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'], temperature=0.7, top_p=0.95, top_k=40)

In [12]:
temperature = 0.5
answer = vector_embeddings.generate_text(prompt=prompt,
                            text_model=text_model,
                            candidate_count=3,
                            temperature=temperature,
                            max_output_tokens=1000)

In [13]:
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

Candidate 0: To shift gears, simply move the shift lever to the desired position. Park, Reverse, Neutral, Drive, and Low.

Candidate 1: To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in the gas pedal. Drive: This position is used to drive forward. Low: This position is used for low speeds.

Candidate 2: To shift gears, simply move the shift lever to the desired position. There are 5 positions: Park, Reverse, Neutral, Drive, and Low.

